In [1]:
! pip install --upgrade joblib==1.1.1
! pip install sentence-transformers
! pip install fugashi
! pip install unidic-lite
! pip install ipadic
! pip install sentencepiece
! pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=41bd038ea9a74b5eec4f5d8cea905d9810fc16b73e3863c9b1a32a947c0228ae
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence-transformers
Looking in indexes: https://pypi.o

# Preprocessing text

In [2]:
file_path = "/content/drive/MyDrive/研究開発/プロジェクト/class_evaluation/Kakegawa Higashi Junior High School/中2社会/中2社会.xlsx"
new_file = file_path.replace('.xlsx', '')+"_clustered.xlsx"

In [3]:
import shutil
shutil.copy(file_path, new_file)

'/content/drive/MyDrive/研究開発/プロジェクト/class_evaluation/Kakegawa Higashi Junior High School/中2社会/中2社会_copy.xlsx'

In [4]:
import pandas as pd

data = pd.read_excel(new_file, sheet_name=None, skiprows=7) #Read multiple sheets using sheet_name = None
len(data)

3

In [ ]:
print(data.keys())

dict_keys(['01_社会2年1組', '02_社会2年3組', '03_社会2年6組', '01_社会2年1組_clustered', '02_社会2年3組_clustered', '03_社会2年6組_clustered'])


In [5]:
import copy
df1 = copy.deepcopy(data)

In [6]:
for x in df1:
  df1[x] = df1[x].rename(columns={df1[x].columns[0]: "Answer", df1[x].columns[1]:"First Thoughts", df1[x].columns[2]:"Post Thoughts"})

In [7]:
df1[list(df1.keys())[0]].tail()

,Answer,First Thoughts,Post Thoughts
27,2128,ヨーロッパとアメリカは植民地が多かったから。（支配しているところが多い・強いから）,産業革命が起こり色々なことが、人々にいい影響を与えたけれどそれによって悪い影響[格差や貧困な...
28,2129,NaN,ヨーロッパから多くの移民を受け入れて人口が増え、技術が向上して機械科が進み農業や工業が発達し...
29,2130,NaN,NaN
30,2131,NaN,NaN
31,2132,勝つために色んなことしていた,国民の人たちを強くなったり、産業も新しくなると思いました。


Normalize

In [8]:
import neologdn
def normalize_x(x):
  if type(x)!=str:
    x = '' # Convert to blank

  x = neologdn.normalize(x)
  return x

In [9]:
for x in df1:
  df1[x]['First Thoughts'] = df1[x]['First Thoughts'].apply(lambda x: normalize_x(x))
  df1[x]['Post Thoughts'] = df1[x]['Post Thoughts'].apply(lambda x: normalize_x(x))

In [12]:
df1[list(df1.keys())[0]].tail()

,Answer,First Thoughts,Post Thoughts
27,2128,ヨーロッパとアメリカは植民地が多かったから。(支配しているところが多い・強いから),産業革命が起こり色々なことが、人々にいい影響を与えたけれどそれによって悪い影響[格差や貧困な...
28,2129,,ヨーロッパから多くの移民を受け入れて人口が増え、技術が向上して機械科が進み農業や工業が発達し...
29,2130,,
30,2131,,
31,2132,勝つために色んなことしていた,国民の人たちを強くなったり、産業も新しくなると思いました。


Remove stopwords and numericals

In [10]:
!wget https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.txt

--2023-03-27 00:57:55--  https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1006 [text/plain]
Saving to: ‘stopwords-ja.txt’

stopwords-ja.txt    100%[===================>]    1006  --.-KB/s    in 0s      

2023-03-27 00:57:55 (48.3 MB/s) - ‘stopwords-ja.txt’ saved [1006/1006]



In [11]:
temp = open('stopwords-ja.txt','r').read().split('\n')
my_stopwords = temp
remove_digits = str.maketrans('0123456789', '%%%%%%%%%%')

def remove_mystopwords(sentence):
    tokens = sentence.split(" ")
    tokens_filtered= [word for word in tokens if not word in my_stopwords]
    return (" ").join(tokens_filtered)

def remove_unness_character(x):
  x = remove_mystopwords(x)
  x = x.translate(remove_digits)
  return x


In [13]:
df2 = copy.deepcopy(df1)
for x in df2:
  df2[x]['First Thoughts'] = df2[x]['First Thoughts'].apply(lambda x: remove_unness_character(x))
  df2[x]['Post Thoughts'] = df2[x]['Post Thoughts'].apply(lambda x: remove_unness_character(x))

In [ ]:
df1[list(df1.keys())[0]].tail() == df2[list(df2.keys())[0]].tail()

,Answer,First Thoughts,Post Thoughts
27,True,True,True
28,True,True,True
29,True,True,True
30,True,True,True
31,True,True,True


In [ ]:
df1[list(df1.keys())[0]].tail()

,Answer,First Thoughts,Post Thoughts
27,2128,ヨーロッパとアメリカは植民地が多かったから 支配しているところが多い 強いから,産業革命が起こり色々なことが 人々にいい影響を与えたけれどそれによって悪い影響 格差や貧困な...
28,2129,,ヨーロッパから多くの移民を受け入れて人口が増え 技術が向上して機械科が進み農業や工業が発達し...
29,2130,,
30,2131,,
31,2132,勝つために色んなことしていた,国民の人たちを強くなったり 産業も新しくなると思いました


Removing punctuation marks

In [15]:
import string
punctuation = string.punctuation + '！’”＃＄％＾＆＊（）＿＋＝：；＜＞「」＠『』。、“■・【】●○⌒☆｀´→'
regular_punct = list(punctuation)
def remove_punctuation(text, punct_list=regular_punct):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, ' ')
    return text.strip()

In [16]:
df3 = copy.deepcopy(df2)
for x in df3:
  df3[x]['First Thoughts'] = df3[x]['First Thoughts'].apply(lambda x: remove_punctuation(x))
  df3[x]['Post Thoughts'] = df3[x]['Post Thoughts'].apply(lambda x: remove_punctuation(x))

In [17]:
data_preprocessed = copy.deepcopy(df3)

# Import Model

In [20]:
import transformers
import sentence_transformers
# transformers.BertTokenizer = transformers.BertJapaneseTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers import models, datasets, losses

bert = models.Transformer("sonoisa/sentence-luke-japanese-base-lite")
pooling = models.Pooling(
        bert.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
)

model = SentenceTransformer(modules=[bert, pooling])

Text Encoding

In [ ]:
first_thoughts_clustering = model.encode(data_preprocessed['First Thoughts'].to_list(), show_progress_bar=True)
print(first_thoughts_clustering.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(32, 768)


In [ ]:
post_thoughts_clustering = model.encode(data_preprocessed['Post Thoughts'].to_list(), show_progress_bar=True)
print(post_thoughts_clustering.shape)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(32, 768)


# KMEANS applying

In [21]:
from sklearn.cluster import KMeans
import numpy as np

def apply_kmeans(X_clustering, n_clusters):
  kmeans = KMeans(n_clusters = n_clusters, random_state=0, init='k-means++', n_init=10)
  kmeans.fit(X_clustering)
  return kmeans.labels_

In [22]:
xl = pd.ExcelFile(new_file)
print(xl.sheet_names)
i = 0
for x in data_preprocessed:
  first_thoughts_clustering = model.encode(data_preprocessed[x]['First Thoughts'].to_list(), show_progress_bar=True)
  post_thoughts_clustering = model.encode(data_preprocessed[x]['Post Thoughts'].to_list(), show_progress_bar=True)
  first_thoughts_labels = apply_kmeans(first_thoughts_clustering, n_clusters=6)
  post_thoughts_labels = apply_kmeans(post_thoughts_clustering, n_clusters=6)


  df_first_thoughts = df1[x][['Answer', 'First Thoughts']]
  df_first_thoughts['first_thoughts_group'] = first_thoughts_labels
  df_first_thoughts = df_first_thoughts.sort_values('first_thoughts_group')

  df_post_thoughts = df1[x][['Answer', 'Post Thoughts']]
  df_post_thoughts['post_thoughts_group'] = post_thoughts_labels
  df_post_thoughts = df_post_thoughts.sort_values('post_thoughts_group')

  with pd.ExcelWriter(new_file, mode='a', if_sheet_exists='replace') as writer:  
    df_first_thoughts.to_excel(writer, sheet_name=xl.sheet_names[i]+'_clustered')
  
  with pd.ExcelWriter(new_file, mode='a', if_sheet_exists='overlay') as writer:  
      df_post_thoughts.to_excel(writer, startcol=7, sheet_name=xl.sheet_names[i]+'_clustered')
  
  i+=1

['01_社会2年1組', '02_社会2年3組', '03_社会2年6組']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
for x in data_preprocessed:
  print(df1[x])

In [ ]:
df1[list(df1.keys())[0]]

,Answer,First Thoughts,Post Thoughts
0,2101,アフリカやヨーロッパで エジプト文明やメソポタミア文明など 早くから文明が発達し ギリシャ帝...,欧米は ピューリタン革命や名誉革命 アメリカの独立戦争 フランス革命や人権宣言など 様々な改...
1,2102,世界に自分の勢力を知らして自分の国だけではなく他の国も統一しようとしたから,元々は国を支配する目的で王国を築いていったけど 国を経済を回すために貿易の取引となる材料の石...
2,2103,一般市民が立ち上がり 絶対王政などの今までの普通を覆した 他の国と戦争みたいなのを初めて 領...,絶対王政に対する革命を人々がしたり 政党政治や普通選挙ができたりし 今までの政治のやり方が変...
3,2104,国王が自己中で それに反対する人が多かったから,この時代では 国王や貴族などの身分の上のものが絶対で 人々はその命令に逆らえない時代だった ...
4,2105,わかりません,イギリスは広大な土地 労働力が整っていて 何より産業革命が起こったことで機械化が進み大幅に工...
5,2106,,王の絶対王政や専制によって反発した議会などがピューリタン革命や名誉革命 フランス革命などの革...
6,2107,人々が発展に向けて行動をしていったから,沢山の革命が起こったから 例えばピューリタン革命では共和政が実現したり 産業革命で貿易が発達...
7,2108,,
8,2109,他の国と仲良くなって 色々な情報を手に入れたから 王様が人気者だったから国民が言うことを聞いた,国王が絶対的な権力を握る絶対王政が始まったり パリや地方で人々が立ち上がり フランス革命が始...
9,2110,,


In [ ]:
first_thoughts_labels = apply_kmeans(first_thoughts_clustering, n_clusters=6)
post_thoughts_labels = apply_kmeans(post_thoughts_clustering, n_clusters=6)
print(first_thoughts_labels)
print(post_thoughts_labels)

[3 3 3 3 4 1 3 1 3 1 3 5 0 2 5 2 5 3 0 1 3 1 5 3 2 2 1 5 1 1 1 3]
[0 2 0 2 4 2 2 1 2 1 1 0 3 2 0 2 0 2 2 4 2 1 2 0 2 0 1 0 4 1 1 5]


In [ ]:
# Export first thoughts to excel
df_first_thoughts = data[['Answer', 'First Thoughts']]
df_first_thoughts['first_thoughts_group'] = first_thoughts_labels
df_first_thoughts = df_first_thoughts.sort_values('first_thoughts_group')
# df_first_thoughts.to_excel('first_thoughts_group.xlsx')

# Export post thoughts to excel
df_post_thoughts = data[['Answer', 'Post Thoughts']]
df_post_thoughts['post_thoughts_group'] = post_thoughts_labels
df_post_thoughts = df_post_thoughts.sort_values('post_thoughts_group')
# df_post_thoughts.to_excel('post_thoughts_group.xlsx')

In [ ]:
df_first_thoughts

In [ ]:
xl = pd.ExcelFile(new_file)
print(xl.sheet_names)  # see all sheet names

['01_社会2年1組', '02_社会2年3組', '03_社会2年6組']


In [ ]:
# Append DataFrame to existing excel file
with pd.ExcelWriter(new_file, mode='a', if_sheet_exists='replace') as writer:  
    df_first_thoughts.to_excel(writer, sheet_name=xl.sheet_names[0]+'_clustered')
  
with pd.ExcelWriter(new_file, mode='a', if_sheet_exists='overlay') as writer:  
    df_post_thoughts.to_excel(writer, startcol=7, sheet_name=xl.sheet_names[0]+'_clustered')


In [ ]:
import copy
b = 4
a = copy.copy(b)
a = 3

print(b)

4
